In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
cd '/content/gdrive/My Drive/ALTEGRAD2020'

/content/gdrive/My Drive/ALTEGRAD2020


In [0]:
!pip install unidecode

     |████████████████████████████████| 245kB 10.2MB/s 


In [0]:
import codecs
import sys
import csv
from unidecode import unidecode
import pandas as pd
import numpy as np
import string
import networkx as nx
import scipy.sparse as sp
import nltk
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('french'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
data_path = "./data/text/text/"
edgelist_path = "./data/edgelist.txt"

##  1. Data Preprocessing

In [0]:
def build_graph():
    '''Function that build a directed weighted graph from the edgelist.txt'''
    G = nx.read_weighted_edgelist(edgelist_path, create_using=nx.DiGraph())
    print("Number of nodes : ", G.number_of_nodes())
    print("Number of edges : ", G.number_of_edges())
    return G

def build_train_test(train_path, test_path):
    """Function that reads the train.csv and returns the train Ids and train labels
        and reads the test.csv and returns the test Ids
    """
    with open(train_path, 'r') as f:
        train_data = f.read().splitlines()
        
    train_hosts = list()
    y_train = list()
    for row in train_data:
        host, label = row.split(",")
        train_hosts.append(host)
        y_train.append(label.lower())
        
    df_train = pd.DataFrame(data= y_train, index = train_hosts, columns= ["class"]).reset_index()
    
    with open(test_path, 'r') as f:
        test_hosts = f.read().splitlines()
    df_test =  pd.DataFrame(data=[] , index = test_hosts, columns= ["class"]).reset_index()
    return df_train, df_test

def write_submission(write_path, test_hosts, model_classes_list, predicted_probas):
    """Function that writes the submission file
  there is a need to be pass it  : 
    - The path of the file to create
    - The test Ids (returned by build_train_test)
    - The classes labels as a list
    - The predicted probas for those class labels (same order)
    """
    with open(write_path, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        model_classes_list.insert(0, "Host")
        writer.writerow(model_classes_list)
        for i,test_host in enumerate(test_hosts):
            lst = predicted_probas[i,:].tolist()
            lst.insert(0, test_host)
            writer.writerow(lst)

def text_from_id(id):
    id = str(id)
    try :
        with codecs.open(data_path+id, 'r', encoding="utf-8") as f:
            text = f.readlines()
    except:
        with codecs.open(data_path+id, 'r', encoding="latin-1") as f:
            text = f.readlines()
    return text



def build_local_test(train_hosts, y_train, size_local_test=.25):
    
    local_train, local_test, local_y_train, local_y_test = train_test_split(train_hosts, y_train,
                                                                            stratify=y_train, 
                                                                            test_size=size_local_test)
    
    return local_train, local_y_train, local_test, local_y_test

def compute_score(predictions, y_true, classes_order):
    dico = {v:k for k, v in enumerate(classes_order)}
    print(dico)
    loss = 0
    for i, cla in enumerate(y_true) :
        loss -= np.log(predictions[i, dico[cla]])
    loss = loss/len(y_true)
    return loss

In [0]:
def join_with_SEP(text):
    """text here is a list of sentences"""
    return " SEP ".join(text)

def clean_page_from(page, rows_that_contain):
    """Removes the rows of 'page' that contain 'rows_that_contain' 
    """
    return [x for x in page if rows_that_contain not in x]

def split_by_SEP(text) :
    """split again by SEP"""
    return  text.split(" SEP ")

def remove_empty_rows(list_of_sentences):
    list_ = [row.strip() for row in list_of_sentences]
    return [row for row in list_ if len(row)>0]

def process_text(text, start_fraction=0, end_fraction=1):
    """Read the text in but keeps the sentences in shape (do not split into tokens) 
    We only read the fraction of the text between start_fraction and end_fraction (we skip the headers of the website 
    and the contacts etc that are in the end
    
    While doing this applies this first preprocessing :
        - removes accents
        - removes the "\n"
        - removes the rows containing the at least one element of the list elements_discarding_the_row
        - handling some weird cases where a website is split on two rows
        - stripping extra spaces from the start and end of each sentence
        - replace appostrophees by space (to delete stopwords further)
    """

    text_ = [unidecode(sentence.replace("\n","").lower()) for sentence in text[int(len(text)*start_fraction):int(len(text)*end_fraction)]]
    
    elements_discarding_the_row = ["tel", "mail", "http", "www", "iframe", "button", 
                               "img", "submit", "lire la suite", "cedex", "html", "css",
                               "cookies","logo", "adresse electronique","jpg","jpeg","png","div","alternate",".fr",".com"]
    
    for e in elements_discarding_the_row :
        text_ = clean_page_from(page=text_, rows_that_contain=e)
    
    text_ = [row for row in text_ if not (("&" in row) & ("=" in row))] # fragmented website
    text_ = [x.replace("'"," ").replace("’"," ") for x in text_ if x != ""]
    text_ = [row.strip() for row in text_]
    
    return text_

def replace_by_special_token(column_df) :
    
    col = column_df.str.replace("(\d{1,2} [a-z]{3,9} \d{4})|(\d{1,2}\s?/\s?\d{1,2}\s?/\s?\d{4})", " date ")
    col = col.str.replace("\d+\s?(euro|euros|eur|EUR)"," prix ")
    col = col.str.replace("\d{1,2}:\d{2}"," heure ")
    return col

def punctuation_by_space(column_df) :
    """ column_df is a column of a dataframe"""
    return column_df.str.replace(r"[{}]".format(string.punctuation+"»"), " ")

def remove_stop_words(text) :
    return " ".join([tok for tok in text.split(" ") if tok not in stop_words])

def remove_single_word_rows(text):
    return [row for row in text if len(row.split())>1]

def remove_single_characters(text):
    return [' '.join( [w for w in row.split() if len(w)>2] ) for row in text]


In [0]:
train_path = "data/train_noduplicates.csv" 
test_path = "data/test.csv"

train_data, test_data = build_train_test(train_path, test_path)

In [0]:
'''test_data['text'] = test_data["index"].apply(text_from_id)
test_data["class_codes"] = pd.Categorical(test_data["class"]).codes
test_data.to_csv('test_data_with_text.csv')'''

test_data = pd.read_csv('test_data_with_text.csv')

In [0]:
'''train_data['text'] = train_data["index"].apply(text_from_id)
train_data["class_codes"] = pd.Categorical(train_data["class"]).codes
train_data.to_csv('train_data_with_text.csv')'''

train_data = pd.read_csv('train_data_with_text.csv')

In [0]:
start_fraction = 0
end_fraction = 1

In [0]:
train_data["text_processed"] = train_data.text.apply(process_text, args=(start_fraction, end_fraction,)) 
train_data["text_processed"] = train_data.text_processed.apply(join_with_SEP)
train_data["text_processed"] = replace_by_special_token(train_data["text_processed"])
train_data["text_processed"] = punctuation_by_space(train_data["text_processed"])
train_data["text_processed"] = train_data.text_processed.apply(remove_stop_words)
train_data["text_processed"] = train_data.text_processed.apply(split_by_SEP)
train_data["text_processed"] = train_data.text_processed.apply(remove_empty_rows)
train_data["text_processed"] = train_data.text_processed.apply(remove_single_characters)
train_data["text_processed_no_single_words"] = train_data.text_processed.apply(remove_single_word_rows)

In [0]:
train_data.to_csv('preprocessed_train_data.csv')

## 2. Model fitting

In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 44.3MB/s 
     |████████████████████████████████| 1.0MB 45.1MB/s 
     |████████████████████████████████| 870kB 65.8MB/s 
     |████████████████████████████████| 3.7MB 63.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=2bd683769aa5ad8ebca11eaeb16f8311b6e840fb857187f543225c594d234595
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import CamembertTokenizer, CamembertModel, CamembertForSequenceClassification
import torch

tokenizer_ = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertModel.from_pretrained('camembert-base')

In [0]:
model.eval();
model.to('cuda');

In [0]:
length = train_data.shape[0] 
for j in range(train_data.shape[0]):
    sys.stdout.write('\r'+str(j)+"/"+str(length))
    target = train_preprocessed["class_codes"].iloc[j]

    txt = ". ".join(train_data.text.iloc[j])
    try :
      tokens = tokenizer_.encode(txt, add_special_tokens=True)
      shape = len(tokens[1:-1])
      new_tokens = []
      for i in range(int(shape/510)+1):
          min_ = min((i+1)*510,shape)
          if min_ == shape :
              L = [tokenizer_.cls_token_id] + tokens[i*510:min_] + [tokenizer_.eos_token_id]
              new_tokens.append(L + [tokenizer_.pad_token_id]*(512 - len(L)))
          else :
              new_tokens.append([tokenizer_.cls_token_id] + tokens[i*510:min_] + [tokenizer_.eos_token_id] )
      with torch.no_grad() :
          new_train_ = model(torch.tensor(new_tokens).cuda())[0][:,0,:]
      del new_tokens
      torch.cuda.empty_cache()
      if j == 0 :
        new_train = new_train_.detach().cpu().numpy().mean(axis=0).reshape(1,-1)
        new_train_target = [target]
      else :
        new_train = np.concatenate((new_train, 
                                    new_train_.detach().cpu().numpy().mean(axis=0).reshape(1,-1)), 
                                   axis=0)
        new_train_target.append(target)
    except :
      new_train = np.concatenate((new_train, np.zeros((1,768))), axis=0)
      new_train_target.extend([target])
new_train = np.array(new_train)

Token indices sequence length is longer than the specified maximum sequence length for this model (2679 > 512). Running this sequence through the model will result in indexing errors


5/1994

Token indices sequence length is longer than the specified maximum sequence length for this model (9598 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (15974 > 512). Running this sequence through the model will result in indexing errors


8/1994

Token indices sequence length is longer than the specified maximum sequence length for this model (5736 > 512). Running this sequence through the model will result in indexing errors


...

1990/1994

Token indices sequence length is longer than the specified maximum sequence length for this model (8377 > 512). Running this sequence through the model will result in indexing errors


1992/1994

Token indices sequence length is longer than the specified maximum sequence length for this model (8437 > 512). Running this sequence through the model will result in indexing errors


1993/1994

In [0]:
train_camembert = pd.DataFrame(new_train)
train_camembert['target'] = new_train_target
train_camembert.to_csv('train_camembert.csv', index=False)

In [0]:
train_camembert = pd.read_csv('train_camembert.csv')

In [0]:
train_camembert.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,target
0,0.004979,0.051084,0.011548,0.144239,-0.054896,-0.091976,-0.079737,0.158545,-0.063696,0.029545,-0.028441,0.035867,-0.028134,0.150745,0.162346,-0.035347,-0.001045,-0.116933,0.137100,-0.052911,0.046483,-0.012581,0.034688,-0.209713,0.126438,-0.194954,-0.005686,0.055490,-0.018563,0.043815,0.024498,-0.170535,0.127708,0.183329,-0.058293,0.028888,0.003619,0.127561,0.032685,-0.005017,...,0.112802,0.088039,0.314132,0.163118,-0.062759,-0.075335,0.011383,0.013771,-0.061509,0.046618,-0.003913,-0.064163,0.053930,0.042424,0.023481,-0.163376,-0.105974,0.039036,-0.039118,-0.163902,0.032288,0.022957,0.030852,-0.239209,0.091170,-0.015870,-0.105228,-0.125584,-0.003099,-0.039667,-0.042257,0.001405,0.040467,0.062530,0.063022,-0.122656,-0.116885,0.005459,0.004883,3
1,-0.008539,0.196301,-0.017056,-0.081347,0.046665,-0.032499,0.080598,0.025480,-0.037952,0.034152,0.106114,0.201544,-0.049484,0.059490,0.099460,-0.075301,0.074141,-0.065830,-0.072313,0.040387,0.025943,-0.075173,-0.108210,-0.191474,0.446738,-0.188546,0.064377,-0.072099,0.023127,-0.110718,-0.130323,-0.093393,-0.063916,0.146621,0.045616,-0.069296,-0.204344,0.036896,0.108239,-0.230537,...,0.113540,-0.105509,0.108981,0.152414,-0.000489,-0.108477,0.171643,-0.018050,-0.030431,-0.080140,-0.007850,0.093457,-0.027985,0.041688,-0.066457,-0.049049,0.087364,0.104405,-0.057261,0.004004,-0.134647,-0.038340,0.125589,-0.069518,0.032810,-0.129408,0.002430,-0.167985,-0.183271,0.027825,-0.048238,-0.067930,0.172509,-0.102032,0.005831,0.000533,-0.095894,-0.107140,-0.166406,2
2,0.014586,0.010158,0.019481,0.139028,-0.069928,-0.046199,-0.043685,0.151316,-0.040847,0.036641,0.029062,0.051212,-0.044985,0.119287,0.193056,-0.060690,-0.070176,-0.121432,0.084575,-0.064783,0.040374,-0.044050,0.091162,-0.182240,0.132107,-0.152883,-0.091903,0.034840,0.024752,-0.013717,-0.032962,-0.166242,0.136357,0.155130,-0.029583,0.049545,-0.060612,0.123955,0.027059,-0.036924,...,0.097372,0.096647,0.300638,0.132141,-0.123732,-0.007762,0.035959,0.007439,-0.066337,-0.012342,-0.004053,-0.046152,0.135284,0.022371,0.024677,-0.114882,-0.071554,0.016102,-0.011974,-0.173243,0.043724,0.024285,0.064807,-0.211291,0.103848,-0.010146,-0.137891,-0.148035,-0.007298,-0.061166,0.022263,-0.050842,0.149352,0.073157,0.062471,-0.108098,-0.133127,0.026859,-0.020756,2
3,0.009449,0.057803,-0.032411,0.181052,-0.023205,-0.077158,-0.088522,0.112174,-0.064386,0.011736,-0.007792,0.100436,0.003390,0.168092,0.165106,-0.055701,-0.022107,-0.087624,0.058423,-0.026992,0.035437,-0.050085,0.067647,-0.218709,0.148364,-0.155343,-0.001270,0.080315,-0.040227,-0.052534,0.019631,-0.110544,0.101820,0.177629,-0.008147,0.044479,0.016035,0.097301,0.023910,-0.030328,...,0.066725,0.087909,0.215426,0.135860,-0.078239,-0.063461,0.016133,0.024904,-0.032214,-0.042499,-0.012666,0.030743,0.090259,0.012712,0.034152,-0.061917,-0.043080,0.021741,-0.033634,-0.210305,0.038935,0.029853,0.004078,-0.209901,0.028864,-0.020685,-0.116714,-0.089824,-0.000962,-0.069954,-0.040098,0.052144,0.014680,0.069520,0.018773,-0.120430,-0.096468,0.036334,0.023615,1
4,-0.018320,0.008598,-0.035451,0.050254,-0.067355,-0.035810,-0.083052,0.092934,-0.036804,0.049422,0.022433,0.036013,0.020027,0.106636,0.179138,-0.093071,0.029542,-0.083329,0.082547,-0.071713,0.012202,-0.013264,0.011300,-0.291337,0.254224,-0.155738,-0.058512,0.057700,0.006700,-0.112091,0.014196,-0.116371,0.065155,0.173727,0.026280,-0.047481,-0.016186,0.131530,0.008204,-0.097958,...,0.096710,0.059853,0.171077,0.128387,-0.065607,-0.015473,0.089697,0.011542,-0.057754,0.037398,-0.001983,0.005053,0.060829,0.016896,0.042919,-0.144442,0.004372,0.075883,-0.037403,-0.142559,-0.002616,-0.017627,0.090729,-0.196299,0.055102,-0.073774,-0.033602,-0.132747,-0.059542,0.019749,-0.002796,-0.067205,0.

In [0]:
X_train = train_camembert.iloc[:,:-1]
y_train = train_camembert.target

In [0]:
idx_exceptions = np.where(X_train.mean(axis=1) == 0.)[0]

In [0]:
for idx in idx_exceptions: 
    train_camembert.iloc[idx] = train_camembert.iloc[idx + 5] 

In [0]:
import logging
logging.getLogger("pytorch_transformers.tokenization_utils").setLevel(logging.ERROR)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [0]:
def loglikelihood_score(y_true, predictions, classes_order):
    dic = {v:k for k, v in enumerate(classes_order)}
    loss = 0
    for i, cls in enumerate(y_true) :
        loss -= np.log(predictions[i, dic[cls]])
    loss = loss/len(y_true)
    return loss

In [0]:
X_train = X_train.values
y_train = y_train.values
grid={"C":np.logspace(-1,3, num = 30)}

logreg = LogisticRegression(solver='lbfgs',  multi_class='auto', max_iter=25000, n_jobs=-1)

classes_order = LogisticRegression(solver='lbfgs',  multi_class='auto').fit(X_train[:, :2], y_train).classes_
score_function = make_scorer(loglikelihood_score, greater_is_better=False, classes_order=classes_order, needs_proba=True)

logreg_cv = GridSearchCV(logreg,grid,cv=3, verbose=3, n_jobs=-1, scoring=score_function)


logreg_cv.fit(X_train, y_train)

print(logreg_cv.best_params_)
print('Score on the local test : ', logreg_cv.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  4.8min finished


{'C': 1.743328822199988}
Score on the local test :  -1.224781800432518


In [0]:
write_submission("./pocoloco.csv", 
                 list(test_data["index"]), 
                 model_classes_list=classes, 
                 predicted_probas=lr.predict_proba(test_camembert))

## 3. Test phase

In [0]:
length = test_data.shape[0] 
for j in range(test_data.shape[0]):
    sys.stdout.write('\r'+str(j)+"/"+str(length))
    target = test_data["class_codes"].iloc[j]

    txt = ". ".join(test_data.final_text.iloc[j])
    try :
      tokens = tokenizer_.encode(txt, add_special_tokens=True)
      shape = len(tokens[1:-1])
      new_tokens = []
      for i in range(int(shape/510)+1):
          min_ = min((i+1)*510,shape)
          if min_ == shape :
              L = [tokenizer_.cls_token_id] + tokens[i*510:min_] + [tokenizer_.eos_token_id]
              new_tokens.append(L + [tokenizer_.pad_token_id]*(512 - len(L)))
          else :
              new_tokens.append([tokenizer_.cls_token_id] + tokens[i*510:min_] + [tokenizer_.eos_token_id] )
      with torch.no_grad() :
          new_test_ = model(torch.tensor(new_tokens).cuda())[0][:,0,:]
      del new_tokens
      torch.cuda.empty_cache()
      if j == 0 :
        new_test = new_test_.detach().cpu().numpy().mean(axis=0).reshape(1,-1)
        new_test_target = [target]
      else :
        new_test = np.concatenate((new_test, 
                                    new_test_.detach().cpu().numpy().mean(axis=0).reshape(1,-1)), 
                                   axis=0)
        new_test_target.append(target)
    except :
      new_test = np.concatenate((new_test, np.zeros((1,768))), axis=0)
      new_test_target.extend([target])
new_test = np.array(new_test)

In [0]:
test_camembert = pd.DataFrame(new_test)
test_camembert.to_csv('test_camembert.csv', index=False)

In [0]:
test_camembert = pd.read_csv('test_camembert.csv')

In [0]:
idx_exceptions = np.where(test_camembert.mean(axis=1) == 0.)[0]
idx_exceptions

array([ 76, 133, 289, 304, 349, 437, 525])

In [0]:
for i in idx_exceptions:
   test_camembert.iloc[i] = test_camembert.iloc[i - 1]

In [0]:
preds = logreg_cv.best_estimator_.predict_proba(test_camembert)

In [0]:
classes = ['business/finance',
 'education/research',
 'entertainment',
 'health/medical',
 'news/press',
 'politics/government/law',
 'sports',
 'tech/science']

In [0]:
import lightgbm as lgb

In [0]:
grid={
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    }

gbm = lgb.LGBMClassifier(boosting_type= 'gbdt')

classes_order = lgb.LGBMClassifier(boosting_type= 'gbdt', multi_class = 'auto').fit(X_train[:, :2], y_train).classes_
score_function = make_scorer(loglikelihood_score, greater_is_better=False, classes_order=classes_order, needs_proba=True)

gbm_cv = GridSearchCV(gbm,grid,cv=3, verbose=3, n_jobs=-1, scoring=score_function)

gbm_cv.fit(X_train, y_train)

print(gbm_cv.best_params_)
print('Score on the local test : ', gbm_cv.best_score_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 31.6min
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed: 32.2min finished


{'boosting_type': 'gbdt', 'colsample_bytree': 0.66, 'learning_rate': 0.005, 'n_estimators': 40, 'num_leaves': 16, 'objective': 'binary', 'random_state': 501, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.7}
Score on the local test :  -1.6917896161997852


In [0]:
gbm_preds = gbm_cv.best_estimator_.predict_proba(test_camembert)

In [0]:
write_submission("./submission.csv", 
                 list(test_data["index"]), 
                 model_classes_list=classes, 
                 predicted_probas=preds)

## 4. Camembert Classification

In [0]:
from transformers import CamembertForSequenceClassification, AdamW, CamembertConfig

# Load CamembertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base", # Use the 12-layer CamemBERT model
    num_labels = 8, # The number of output labels-- 8 for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [0]:

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)